## <B>Statistics/P-value/Cohen's d statistical validation after normality check (all data)</B>

In [ ]:
import pandas as pd
from scipy import stats
import warnings

# 경고를 무시하도록 설정
warnings.filterwarnings("ignore")

# 파일 경로 및 시트 이름
file_path = '실험 결과.xlsx'
sheet_name = '시선 영역'

# 데이터 읽기
df = pd.read_excel(file_path, sheet_name=sheet_name)
df_subset = df.iloc[14:39]

# 결과 저장용 리스트
results = []

# 1부터 5까지 반복하여 각 원본영상과 블러영상 처리
for i in range(1, 6):
    column1_name = f'원본영상{i}'
    column2_name = f'블러영상{i}'

    # 각 칼럼 선택 및 결측치 제거
    column1 = df_subset[column1_name].dropna()
    column2 = df_subset[column2_name].dropna()

    # 데이터 크기 확인
    if len(column1) == 0 or len(column2) == 0:
        print(f"{column1_name} 또는 {column2_name} 칼럼에 유효한 데이터가 없습니다.")
        continue

    # 정규성 검사 (Shapiro-Wilk 테스트)
    shapiro1_stat, shapiro1_p = stats.shapiro(column1)
    shapiro2_stat, shapiro2_p = stats.shapiro(column2)

    # 데이터 정규성 여부
    column1_normal = shapiro1_p > 0.05
    column2_normal = shapiro2_p > 0.05

    if column1_normal and column2_normal:
        # 데이터가 정규성을 만족하면 T-검정 수행
        t_stat, t_p_value = stats.ttest_rel(column1, column2)

        # Cohen's d 계산
        diff = column1 - column2
        cohen_d = diff.mean() / diff.std(ddof=1)  # 표준 편차는 ddof=1로 설정

        # p-value 조건에 따라 처리
        if t_p_value < 0.001:
            p_value_str = "p<0.001"
        else:
            p_value_str = f"p={t_p_value:.3f}"

        # 결과 저장
        results.append({
            'Comparison': f'{column1_name} vs {column2_name}',
            'Test': 'T-test',
            'Statistic': round(t_stat, 3),
            'p-value': p_value_str,
            "Cohen's d": round(cohen_d, 3)
        })

        # 결과 출력
        print(f"\n{column1_name} vs {column2_name}")
        print(f"T={t_stat:.3f}, {p_value_str}, d={cohen_d:.3f}")

    else:
        # 데이터가 정규성을 만족하지 않으면 윌콕슨 검정 수행
        wilcoxon_stat, wilcoxon_p_value = stats.wilcoxon(column1, column2)

        # Wilcoxon의 Cohen's d 계산
        diff = column1 - column2
        cohen_d = diff.mean() / diff.std(ddof=1)  # 표준 편차는 ddof=1로 설정

        # p-value 조건에 따라 처리
        if wilcoxon_p_value < 0.001:
            p_value_str = "p<0.001"
        else:
            p_value_str = f"p={wilcoxon_p_value:.3f}"

        # 결과 저장
        results.append({
            'Comparison': f'{column1_name} vs {column2_name}',
            'Test': 'Wilcoxon',
            'Statistic': round(wilcoxon_stat, 3),
            'p-value': p_value_str,
            "Cohen's d": round(cohen_d, 3)
        })

        # 결과 출력
        print(f"\n{column1_name} vs {column2_name}")
        print(f"W={wilcoxon_stat:.3f}, {p_value_str}, d={cohen_d:.3f}")

# 최종 결과 요약 출력
summary_df = pd.DataFrame(results)
print("\nSummary of Results:")
print(summary_df)

## <B>Statistics/P-value/Cohen's d statistical validation after normality check (OriginalFirst/BlurredFirst data)</B>

In [ ]:
import pandas as pd
from scipy import stats
from scipy.stats import wilcoxon
import warnings
import numpy as np

# 경고를 무시하도록 설정
warnings.filterwarnings("ignore")

# 파일 경로 및 시트 이름
file_path = '실험 결과.xlsx'
sheet_name = '시선 영역'

# 데이터 읽기
df = pd.read_excel(file_path, sheet_name=sheet_name)
df_subset = df.iloc[:39]

# 결과 저장용 리스트
results = []

# 1부터 5까지 반복하여 원본영상과 블러영상에 대해 처리
for i in range(1, 6):
    column1_name = f'원본영상{i}'  # 원본영상 칼럼 이름
    column2_name = f'블러영상{i}'  # 블러영상 칼럼 이름
    
    # 각 칼럼 선택 및 결측치 제거
    column1 = df_subset[column1_name].dropna()
    column2 = df_subset[column2_name].dropna()
    
    # 숫자로 변환하고, 변환 불가 값은 제거
    column1 = pd.to_numeric(column1, errors='coerce').dropna()
    column2 = pd.to_numeric(column2, errors='coerce').dropna()

    # 데이터 크기 확인
    if len(column1) == 0 or len(column2) == 0:
        print(f"{column1_name} 또는 {column2_name} 칼럼에 유효한 데이터가 없습니다.")
        continue

    # 홀수 행, 짝수 행 나누기
    column1_odd = column1.iloc[::2]  # 홀수 행
    column1_even = column1.iloc[1::2]  # 짝수 행

    column2_odd = column2.iloc[::2]  # 홀수 행
    column2_even = column2.iloc[1::2]  # 짝수 행

    for (group_name, col1, col2) in [("홀수", column1_odd, column2_odd), ("짝수", column1_even, column2_even)]:
        if len(col1) == len(col2) and len(col1) > 0:
            # 정규성 검사 (Shapiro-Wilk 테스트)
            shapiro1_stat, shapiro1_p = stats.shapiro(col1)
            shapiro2_stat, shapiro2_p = stats.shapiro(col2)

            # 데이터 정규성 여부
            col1_normal = shapiro1_p > 0.05
            col2_normal = shapiro2_p > 0.05

            if col1_normal and col2_normal:
                # 데이터가 정규성을 만족하면 T-검정 수행
                t_stat, t_p_value = stats.ttest_rel(col1, col2)

                # Cohen's d 계산
                diff = col1 - col2
                cohen_d = diff.mean() / diff.std(ddof=1)

                # p-value 포맷팅
                formatted_p = "p<0.001" if t_p_value < 0.001 else f"p={t_p_value:.3f}"

                # 결과 저장 및 출력
                results.append({
                    'Comparison': f'{column1_name} vs {column2_name} ({group_name})',
                    'Test': 'T-test',
                    'Statistic': round(t_stat, 3),
                    'p-value': formatted_p,
                    "Cohen's d": round(cohen_d, 3)
                })

                print(f"\n{column1_name} vs {column2_name} ({group_name})")
                print(f"T={t_stat:.3f}, {formatted_p}, d={cohen_d:.3f}")
            else:
                # 데이터가 정규성을 만족하지 않으면 Wilcoxon 검정 수행
                wilcoxon_stat, wilcoxon_p_value = wilcoxon(col1, col2)

                # Wilcoxon의 Cohen's d 계산
                diff = col1 - col2
                cohen_d = diff.mean() / diff.std(ddof=1)

                # p-value 포맷팅
                formatted_p = "p<0.001" if wilcoxon_p_value < 0.001 else f"p={wilcoxon_p_value:.3f}"

                # 결과 저장 및 출력
                results.append({
                    'Comparison': f'{column1_name} vs {column2_name} ({group_name})',
                    'Test': 'Wilcoxon',
                    'Statistic': round(wilcoxon_stat, 3),
                    'p-value': formatted_p,
                    "Cohen's d": round(cohen_d, 3)
                })

                print(f"\n{column1_name} vs {column2_name} ({group_name})")
                print(f"W={wilcoxon_stat:.3f}, {formatted_p}, d={cohen_d:.3f}")
        else:
            print(f"{column1_name}의 {group_name} 데이터 크기가 일치하지 않아 검정을 수행할 수 없습니다.")

# 최종 결과 요약 출력
summary_df = pd.DataFrame(results)
print("\nSummary of Results:")
print(summary_df)